# User scraping 

- 강남역 맛집 리뷰 수가 많은 유저의 myplace를 크롤링 (목표 user 1000명)

- user_profile_df : User의 아이디, 리뷰수, 팔로워, url을 저장함
- user_df : '아이디', '리뷰', '팔로워', '매장명', '카테고리', '주소', '리뷰 내용', '세부정보', '태그', '방문일자', '재방문횟수' 를 저장함


In [5]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.action_chains import ActionChains

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  

from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import os
from datetime import datetime

import pandas as pd
from selenium.webdriver.common.by import By

### 함수 

In [6]:
# 정규 표현식으로 이모티콘, 특수문자, 아스키코드 제거
def remove_special_characters(text):
    pattern = r'[^\w\s]|_'
    result = re.sub(pattern, '', text)
    return result

In [7]:
# User의 아이디, 리뷰수, 팔로워, url을 가져오는 함수 

def user_profile(url) :
    # 주소 이동

    global driver
    driver.get(url)
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    user_element = {}

    # User ID 찾기
    user_id_element = soup.find('button', class_='wTaI4v _2kK3N- _2we3hB')
    user_element['아이디'] = user_id_element.text if user_id_element else None

    # User의 인기도 : 리뷰수, 팔로워 찾기 
    user_popularity = soup.find_all('button', class_='wTaI4v _15qVKh')
    
    for element in user_popularity:
        em_tag = element.find('em')
        if em_tag:
            key = element.text.replace(em_tag.text, '').strip()
            value = int(re.sub('[^0-9]', '', em_tag.text))  # 쉼표 제거 후 변환
            user_element[key] = value
    
    # User URL
    user_element['url'] = url

    print(user_element)

    return user_element

def change_date_format(날짜) :
    #  부분 추출 (년월일까지)
    date_part = ""
    for i in 날짜 :
        date_part += i
        date_part += ','
    date_part = date_part.split('\n')[1]

    # 일자, 요일, 방문일수 분류하기 
    date_part = date_part.split(',')

    # '21.1.3.'에서 숫자 부분 추출
    revisit_match = re.search(r'(\d+\.\d+\.\d+)\.', date_part[1])
    if revisit_match:
        revisit_str = revisit_match.group(1)
        # 방문일수 정수로 변환
        revisit = int(revisit_str.replace('.', ''))
    else:
        revisit = 0  # 또는 다른 기본값으로 설정

    # 일자 타입 변경하기 
    date = date_part[0]
    weekday = date[-3:]
    match = re.findall(r'(\d+)년 (\d+)월 (\d+)일', date) # 타입 바꾸기 
    
    if match:
        year, month, day = map(int, match[0])

        # 날짜 객체로 변환
        date_object = datetime(2000 + year, month, day) 
        formatted_date = date_object.strftime('%Y-%m-%d')
    else:
        formatted_date = ''  # 또는 다른 기본값으로 설정
    
    return formatted_date, weekday, revisit


In [63]:
# "내용 더보기" 텍스트를 포함하는 버튼 찾기

def click_read_more_button(driver):
    buttons = driver.find_elements(By.XPATH, '//button[contains(.,"내용 더보기")]')
    if buttons:
        first_button = buttons[0]
        first_button.click()




In [65]:
# 유저 리뷰 가져오는 함수 
def find_user_data(page_down,driver) : 

 
    # data를 담을 빈 리스트 선언
    data_list = [] 

    # # 버튼을 클래스로 찾아 리스트로 가져오기
    # buttons = driver.find_elements(By.CLASS_NAME, '_3P-5HQ')
    # if buttons:
    #     first_button = buttons[0]
    #     first_button.click()

    # 맨 처음 게시물 클릭 
    button = driver.find_element(By.CLASS_NAME, '_3P-5HQ')
    button.click()
    time.sleep(5)

    html = driver.page_source
    soup = BeautifulSoup(driver.page_source, 'html.parser')


    # 페이지 아래로 내리기 
    for _ in range(page_down):

        # '내용더보기' 버튼들 찾고 죄다 클릭해두기  
        try :
            click_read_more_button(driver)
        except :
            pass 
                
        # 다음 페이지로 잘 넘어갔는지 기다리기 
        wait = WebDriverWait(driver, 10)
        user_review_elements = wait.until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, '_27tH92')))
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN) 


        # 리뷰에서 정보 가져오기
        for reviews_elements in user_review_elements:
            i = 0 # 카테고리와 주소 구분할 때 사용
            restaurant_elements=reviews_elements.find_elements(By.CLASS_NAME, '_1QGRWW')

            if bool(restaurant_elements): # 정보가 없는 경우가 있음

                # 매장명 찾기
                restaurant_name = restaurant_elements[0].text  

        #<--------------------------------------------------------------------------------------------------->
                # 카테고리와 주소 찾기 
                category_location_elements = reviews_elements.find_elements(By.CLASS_NAME, '_2vBfgu')
                i = 0  # i 초기화
                for category_location_element in category_location_elements:
                    category_location_soup = BeautifulSoup(category_location_element.get_attribute('outerHTML'), 'html.parser')
                
                    # span 태그 안에 있는 텍스트 가져오기
                    span_elements = category_location_soup.find_all('span', class_='wzFIfJ')
                    category = span_elements[0].text if span_elements and len(span_elements) > 0 else '없음'
                    location = span_elements[1].text if span_elements and len(span_elements) > 1 else '없음'
                #<--------------------------------------------------------------------------------------------------->

                    review_elements = reviews_elements.find_elements(By.CLASS_NAME,'z0t_8b')
                    try : 
                        review_text = review_elements[0].text # 리뷰 본문      
                        review_text = remove_special_characters(review_text)
                        sub_info_elements = reviews_elements.find_elements(By.CLASS_NAME,'_1tkuel') # 세부정보
                        sub_info = sub_info_elements[0].text if sub_info_elements else None
                    except :
                        review_text = '리뷰 없음'
                        sub_info = ''
                #<--------------------------------------------------------------------------------------------------->
                    # tag 찾기 
                    reactions_elements = reviews_elements.find_elements(By.CLASS_NAME, 'COw42b')  # 리엑션 리스트
                    command = False

                    for x in reactions_elements:
                        if x.get_attribute("role"):  # 리액션 숨김 없애기
                            try:
                                # 요소가 클릭 가능할 때까지 대기
                                wait = WebDriverWait(driver, 10)
                                reaction_element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'COw42b')))
                                
                                # JavaScript를 사용하여 요소 클릭
                                driver.execute_script("arguments[0].click();", reaction_element)
                                
                                command = True
                            except:
                                continue

                    if command:
                        # 리엑션 리스트 재탐색
                        reactions_elements = reviews_elements.find_elements(By.CLASS_NAME, 'COw42b')

                    # 리스트를 벗김
                    reactions = list(map(lambda x: x.text, reactions_elements))
                    reactions = str(reactions).replace('[', '').replace(']', '')                    

                #<--------------------------------------------------------------------------------------------------->
                    # 방문 일자, 재방문 이력 찾기
                    date_info = reviews_elements.find_element(By.CLASS_NAME, '_15xwjO .hol3Ic').find_elements(By.CLASS_NAME,'_3nNYBi')
                    date=[x.text for x in date_info]
                    day,weekday,revisit = change_date_format(date) # 함수 사용

                    global user_data

                    # 정보 추가
                    data_dict = {
                        '아이디' : user_data['아이디'],
                        '매장명': restaurant_name,
                        '카테고리': category,
                        '주소' : location,
                        '리뷰 내용': review_text,
                        "세부정보" : sub_info,
                        '태그': reactions,
                        '방문일자': day,
                        '요일' : weekday,
                        '재방문횟수':revisit
                    }
                    data_list.append(data_dict)

        user_df = pd.DataFrame(data_list)
        user_df = user_df.drop_duplicates()

    return user_df



In [9]:
url_list = pd.read_excel('data/4_1_naver_user_크롤링.xlsx')
url_list

,검색어,아이디,url
0,"언주로168길 33, 런던베이글뮤지엄",PoliPoli,https://m.place.naver.com/my/5b9ba11bd551b9e30...
1,"언주로168길 33, 런던베이글뮤지엄",Ms piggy,https://m.place.naver.com/my/5b9c2bbb461a89e6d...
2,"언주로168길 33, 런던베이글뮤지엄",PoliPoli,https://m.place.naver.com/my/5b9ba11bd551b9e30...
3,"영동대로 513, 에그슬럿",이랴챠챠,https://m.place.naver.com/my/5df49f3e8f87a842b...
4,"영동대로 513, 에그슬럿",빙그레7955,https://m.place.naver.com/my/5e58afd88f87a842b...
...,...,...,...
214,광평로 280 버거킹수서점,황금들녘78,https://m.place.naver.com/my/5d5e60f18f87a842b...
215,광평로 280 버거킹수서점,솔솔딸둥이네,https://m.place.naver.com/my/5e400e068f87a842b...
216,"강남대로98길 14, 알부자",쑤9249,https://m.place.naver.com/my/5efdee1d0765f4dd1...
217,"강남대로98길 14, 알부자",퀸지,https://m.place.naver.com/my/5b9b228841fc103c8...


### url에 들어가서 유저 정보와 리뷰 저장

In [66]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install()) 

# driver = webdriver.Chrome()



# 스크롤 횟수 
page_down = 3

# 데이터프레임 선언
total_user_df =  pd.DataFrame()
user_profile_df = pd.DataFrame(columns=['아이디', '리뷰', '팔로워', '팔로잉', 'url'])

# 반복해서 url 정보 가져오기 
for url in tqdm(url_list['url'], desc='진행 중', position=0, leave=True):
    try: 
        # User의 아이디, 리뷰수, 팔로워를 가져오는 함수  
        user_data = user_profile(url)
        
        # 데이터프레임으로 만들기
        user_data_df = pd.DataFrame([user_data])
        user_profile_df = pd.concat([user_profile_df, user_data_df], ignore_index=True)
        
        # User review 정보를 가져오는 함수
        user_df = find_user_data(page_down, driver)

        # User review 정보 저장 
        total_user_df = pd.concat([total_user_df, user_df], ignore_index=True)

    except Exception as e:
        print(f"에러 메시지: {str(e)}")
        continue


진행 중:   0%|          | 0/219 [00:00<?, ?it/s]

{'아이디': None, 'url': 'https://m.place.naver.com/my/5b9ba11bd551b9e3094f6329/review?v=2'}


진행 중:   0%|          | 1/219 [00:03<12:01,  3.31s/it]

에러 메시지: Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3P-5HQ"}
  (Session info: chrome=123.0.6312.106)

{'아이디': 'Ms piggy', '리뷰': 337, '팔로워': 96, '팔로잉': 16, 'url': 'https://m.place.naver.com/my/5b9c2bbb461a89e6d32f3a69/review?v=2'}


진행 중:   1%|          | 2/219 [00:31<1:04:01, 17.70s/it]

...

{'아이디': 'Good Luck37', '리뷰': 1622, '팔로워': 0, '팔로잉': 0, 'url': 'https://m.place.naver.com/my/5c775302c018e29cdc732e67/review?v=2'}


진행 중:   4%|▎         | 8/219 [01:54<51:50, 14.74s/it]

{'아이디': '우유져아54', '리뷰': 258, '팔로워': 0, '팔로잉': 0, 'url': 'https://m.place.naver.com/my/5bd42eacde79f0cc92ac38c7/review?v=2'}


진행 중:   4%|▎         | 8/219 [02:09<56:56, 16.19s/it]


KeyboardInterrupt: 

test 

In [11]:
user_profile_df

,아이디,리뷰,팔로워,팔로잉,url


In [12]:
total_user_df

""


In [ ]:
# 유저의 리뷰
total_user_df.to_excel('./data/4_2_naver_myPlace_크롤링_user_df_test.xlsx', index=False)

# 유저의 정보 
user_profile_df.to_excel('./data/4_2_naver_myPlace_크롤링_user_profile_df_test.xlsx', index=False)